In [1]:
import json
from google.cloud import storage
bucket_name = 'ir-project-bucket-313191645-201013310' 
storage_client = storage.Client()
BUCKET = storage_client.get_bucket(bucket_name)


# get the blob
blob = BUCKET.get_blob('json_files/queries_train.json')
# load blob using json
queries = json.loads(blob.download_as_string())

In [2]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [5]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://b47d-34-72-5-226.ngrok.io'

qs_res = []
for q, true_wids in queries.items():
    duration, ap = None, None
    t_start = time()
    try:
        res = requests.get(url + '/search_body', {'query': q}, timeout=35)
        duration = time() - t_start
        if res.status_code == 200:
            pred_wids, _ = zip(*res.json())
            ap = average_precision(true_wids, pred_wids)
    except:
        print(q + ': not good...')
    qs_res.append((q, duration, ap))

print('Finish')

In [10]:
print(qs_res)
print('MAP@40: ' + str(sum(i[2] for i in qs_res)/30))
print('AVG run time: ' + str(sum(i[1] for i in qs_res)/30))

[('python', 12.294742822647095, 0.168), ('data science', 7.939196348190308, 0.091), ('migraine', 7.624596118927002, 0.234), ('chocolate', 7.866066217422485, 0.159), ('how to make pasta', 7.86054253578186, 0.277), ('Does pasta have preservatives?', 7.693466901779175, 0.0), ('how google works', 7.893348693847656, 0.227), ('what is information retrieval', 7.770978212356567, 0.258), ('NBA', 7.712854862213135, 0.135), ('yoga', 7.680114507675171, 0.522), ('how to not kill plants', 7.916727781295776, 0.111), ('masks', 7.842602491378784, 0.216), ('black friday', 7.76887321472168, 0.0), ('why do men have nipples', 7.8874571323394775, 0.078), ('rubber duck', 7.925971508026123, 0.0), ('michelin', 7.7022483348846436, 0.143), ('what to watch', 7.773510217666626, 0.067), ('best marvel movie', 7.8394997119903564, 0.063), ('how tall is the eiffel tower', 8.07307481765747, 0.236), ('where does vanilla flavoring come from', 8.160261869430542, 0.069), ('best ice cream flavour', 8.065742015838623, 0.09), 